<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-логики-парсера" data-toc-modified-id="Описание-логики-парсера-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание логики парсера</a></span></li><li><span><a href="#Полный-код-парсера-и-сбор-данных" data-toc-modified-id="Полный-код-парсера-и-сбор-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Полный код парсера и сбор данных</a></span></li></ul></div>

# Парсер данных для Liveinternet

В рамках одного из исследований потребовалось собрать данные о поисковых запросах для того, чтобы оценить существующую потребность в новом продукте.  
Исследование проводилось по состоянию на апрель 2023 года. В качестве основного источника данных был выбран сайт Liveinternet, поскольку он позволяет собрать полную статистику по ключевым фразам, а не только наиболее частотные ключи. Полученные данные также сопоставлялись с данными из других источников, в этом проекте присутствует только парсер для сайта Liveinternet.  

## Описание логики парсера

На сайте Liveinternet есть статистика по запросам на различные тематики. На момент сбора статистики в этом источнике есть данные со 146 922 сайтов. Статистика разбита по тематическим категориям. Для целевого исследования могла подойти категория Софт, но там собраны запросы, для которых система смогла однозначно определить тематику - ПО. Поэтому было решено собрать данные в целом, поскольку нужные нам запросы могут и не попасть в какую-либо конкретную категорию.
Статистика на сайте расположена по страницам, которых множество в пределах выбранного периода. Нужно выгрузить статистику по поисковым запросам в единый датасет. Потребуется собрать данные за текущий год (2023), предыдущий год (2022), а также 2021 год, т.к. ситуация на российском рынке в 2022 году была сильно специфичной.

Импорт библиотек:

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as rq
import datetime as dt
import numpy as np

Для получения нужной статистики подходит отчет по поисковым фразам Liveinternet. Поскольку нужны данные за продолжительный период времени, выберем период - месяц и суммарные значения, все это настраивается в представлении с помощью переключателей.

Адрес страницы, с которой будут собираться данные:

In [2]:
url = "https://www.liveinternet.ru/stat/ru/queries.html?period=month"

Получаем данные страницы:

In [3]:
df = pd.read_html(url, encoding='utf-8')

In [4]:
print('Всего найдено таблиц:', len(df))

Всего найдено таблиц: 9


Найдем, в какой таблице хранятся нужные данные:

In [5]:
for i,j in enumerate(df):
    print(f'Начало таблицы {i}')
    display(j.head())
    print(f'Конец таблицы {i}')

Начало таблицы 0


,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,посещаемость по времени суток online за неделю...,NaN,Статистика сайта: ru Срезы: dzen_news ru-andro...,NaN,регистрация Статистика сайта: Забыли или не з...,NaN,NaN,NaN
2,NaN,Статистикадля PDA,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Срезы: dzen_news ru-android uc usa ndr mobile ...,Срезы: dzen_news ru-android uc usa ndr mobile ...,NaN,NaN,NaN,NaN,NaN,NaN


Конец таблицы 0
Начало таблицы 1


,0,1
0,NaN,Статистикадля PDA


Конец таблицы 1
Начало таблицы 2


...

Конец таблицы 6
Начало таблицы 7


,0
0,посещаемость по времени суток online за неделю...


Конец таблицы 7
Начало таблицы 8


,0,1
0,Служба поддержки: counter@corp.liveinternet.ru...,NaN


Конец таблицы 8


Статистика по запросам находится в таблице под номером 5, выделим ее в датафрейм `query`.

In [6]:
query = df[5]
print('Количество записей в таблице:', query.shape)

Количество записей в таблице: (15, 8)


В этот момент появилось ограничение по количеству записей. У меня был выбран вариант отображения по 100 записей на странице, но, по умолчанию в этой таблице на сайте Liveinternet показывается только 10 строк, даже если выбрать вариант отображения с большим количеством. Именно это количество записей можем получить за один запрос.

При этом требуется получить все данные за апрель, а это 26 страниц при количестве записей 10 на странице. При этом при переходе на следующую страницу в  адресной строке появляется аббревиатура `&page=2`, где число - это порядковый номер страницы. Поскольку количество страниц известно, используем это для перебора - будем поочередно увеличивать номер на 1. 

Пропишем заготовку адреса без номера страницы и диапазон страниц. В конце строки с адресом будем поочередно добавлять номер страницы из указанного диапазона.

*заготовка адреса страницы без номера  
url_pages = "https://www.liveinternet.ru/stat/ru/queries.html?period=month&page="*

Страницы можно перебирать циклом, но нужно знать, какая цифра будет последней. Если указать фиксированное значение, то это не сработает. Каждый день количество запросов будет увеличиваться, количество страниц также будет расти, а значит, мы будем получать неполные данные.  
Выше среди полученных с сайта таблиц номер страницы не фигурирует, что логично, потому что это также динамический элемент страницы - гиперссылка. Можно было бы подставить в качестве конечного значения диапазона несуществующую страницу 99, но в нашем случае это не подходит. Это сильно замедлит сбор данных, и тогда в датасете задублируются строки, потому что алгоритм будет все время сохранять данные с последней доступной страницы. Научим парсер самостоятельно находить номер последней страницы в представлении.  
Получим код страницы сайта и определим, в каком из элементов лежит номер страницы.

Сделаем get-запрос и определим статус страницы:

In [7]:
r = rq.get(url)
r.status_code

200

Статус 200 означает, что страница доступна, и с нее можно собирать данные. Получим код страницы:

In [8]:
r.text

'<!--DOCTYPE html-->\r\n<!--html  xml:lang="ru" lang="ru" dir="ltr"-->\r\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ru" lang="ru" dir="ltr">\r\n<head>\r\n<title>статистика сайта &quot;Сайты Рунета&quot;</title>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<meta http-equiv="Pragma" content="no-cache">\r\n<meta http-equiv="Expires" content="-1">\r\n<meta http-equiv="Cache-Control" content="no-cache">\r\n<link rel="shortcut icon" href="/i/fav-stat.ico" type="image/x-icon"/>\r\n<style type="text/css">\r\n.vm { color:#f7f7f7; font-size:11px; font-weight:bold;  text-transform:lowercase; text-decoration:none }\r\n.bc1 { background: #508099;}\r\n.bc2 { background: gray; color: white; width: 136px;}\r\n.bc4 { font: bold 16px;}\r\n.bc41 { font: 12px; color: gray;}\r\n.bc42 { font: bold 14px; color: white; background-color: #ff9900; padding-top: 3px;padding-bottom: 3px;padding-left: 3px; padding-right: 3px; text-decoration: none; width: 500px;}\r\n.bc5 { fo

Номера страниц зашиты в этом куске кода:  

`Страницы: <span class="high current">1</span>&nbsp;&nbsp;<a href="queries.html?period=month&page=2" class="high">2</a>&nbsp;<a href="queries.html?period=month&page=3" class="high">3</a>&nbsp;...&nbsp;<a href="queries.html?period=month&page=26" class="high">26</a>&nbsp;<a href="queries.html?period=month&page=27" class="high">27</a>&nbsp;`

На странице сайта Liveinternet всегда отображается номер первой и последней страницы, а также некоторое количество промежуточных. Нас интересует только последняя страница, ее номер нужно передавать в качестве аргумента для `range` при создании диапазона страниц.  
Преобразуем полученный текст в дерево объектов:

In [9]:
soup = bs(r.text, 'html.parser')

In [10]:
soup

<!--DOCTYPE html-->
<!--html  xml:lang="ru" lang="ru" dir="ltr"-->
<html dir="ltr" lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>статистика сайта "Сайты Рунета"</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Cache-Control"/>
<link href="/i/fav-stat.ico" rel="shortcut icon" type="image/x-icon"/>
<style type="text/css">
.vm { color:#f7f7f7; font-size:11px; font-weight:bold;  text-transform:lowercase; text-decoration:none }
.bc1 { background: #508099;}
.bc2 { background: gray; color: white; width: 136px;}
.bc4 { font: bold 16px;}
.bc41 { font: 12px; color: gray;}
.bc42 { font: bold 14px; color: white; background-color: #ff9900; padding-top: 3px;padding-bottom: 3px;padding-left: 3px; padding-right: 3px; text-decoration: none; width: 500px;}
.bc5 { font: bold 11px;}
.bc6o { font: bold 11px; background: gray;

Нужные данные зашиты в тег `<a>` с классом `high`, найдем их все:

In [11]:
soup_pages = soup.find_all('a', class_='high')
soup_pages

[<a class="high" href="queries.html?period=month&amp;page=2">2</a>,
 <a class="high" href="queries.html?period=month&amp;page=3">3</a>,
 <a class="high" href="queries.html?period=month&amp;page=27">27</a>,
 <a class="high" href="queries.html?period=month&amp;page=28">28</a>,
 <a class="high" href="queries.html?period=month&amp;page=2">следующая</a>]

Получили перечень страниц, номера которых видно в представлении на сайте. Нужно выбрать максимальное значение - это и будет последняя страница.  
Извлечем текст из тега `<a>` и соберем его в список:

In [12]:
pages_num = []
for i in soup_pages:
    num = i.contents
    pages_num.append(num)

In [13]:
pages_num

[['2'], ['3'], ['27'], ['28'], ['следующая']]

Нужный нам номер всегда будет предпоследним элементом в списке. Задаем его как границу диапазона страниц:

In [14]:
last_page = pages_num[-2]

Номера страниц были получены как строки, преобразуем номер в числовой формат:

In [15]:
last_page = [int(x) for x in last_page][0]

Подставляем полученный номер в диапазон страниц, не забыв добавить единицу, т.к. нумерация начинается с 0:

In [16]:
pages = list(range(1,last_page+1))

Также нужно учесть то, что по умолчанию на странице отображаются среднесуточные значения. Для большей информативности потребуются суммарные. Это переключатель, но судя по коду страницы он вложен в таблицу просто ссылкой с параметром `total=yes`:  

`<a href="queries.html?period=month&amp;total=yes">суммарные</a>`  

Но зато, если посмотреть на ссылку в адресной строке, то видно, что при выборе варианта "суммарные" адрес ссылки меняется: к ней добавляется `&total=yes`. Добавим такой параметр к нашей заготовке ссылки.

Теперь можно парсить таблицы.  
Создадим пустой датафрейм и поочередно прибавим к нему каждую полученную со страницы таблицу. 

In [17]:
query_stats = pd.DataFrame()

In [18]:
for i in pages:
    url_pages = "https://www.liveinternet.ru/stat/ru/queries.html?period=month&total=yes&page=" + str(i)
    stats = pd.read_html(url_pages, encoding='utf-8')[5]
    query_stats = pd.concat([query_stats,stats], ignore_index=True)

In [19]:
query_stats

,0,1,2,3,4,5,6,7
0,значения:суммарные / среднесуточные,значения:суммарные / среднесуточные,апрель 2023 г.,апрель 2023 г.,март 2023 г.,март 2023 г.,в среднемза 3 месяца,в среднемза 3 месяца
1,NaN,Другие,11065112,92.2%,27391283,89.5%,21787331,91.2%
2,NaN,новости,71069,0.6%,212279,0.7%,174103,0.7%
3,NaN,газета чс последний номер,36029,0.3%,84210,0.3%,99340,0.4%
4,NaN,майл ру,32251,0.3%,81623,0.3%,60865,0.3%
...,...,...,...,...,...,...,...,...
412,NaN,купить apecoin,170,0.0%,0,0.0%,56,0.0%
413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,NaN,сумма выбранных,902,0.0%,5354,0.0%,7765,0.0%
415,NaN,всего,11998065,NaN,30600753,NaN,23890890,NaN


Пока не обращаем внимания на то, что дублируются первые столбцы в таблице. Это произошло из-за того, что на сайте в первом столбце расположен элемент выбора - флаг, соответственно он не распознается. Этот столбец удалим позже, когда соберем все данные. Также на каждой странице внизу выводилась сумма выбранных - это количество запросов, которые были отмечены активным флагом, и всего - это общее количество запросов на странице. Такие строки также нужно будет удалить, как и полностью пустые строки. Это будет сделано на этапе предобработки.

Получили данные о поисковых запросах за текущий месяц. Для целей исследования понадобятся данные также за более ранние периоды. На сайте возможно только три варианта отображения данных: по дням, по неделям и по месяцам, причем переходить к более ранним периодам можно только последовательно по кнопке.  
Посмотрим, как в коде прописан переход к предущему месяцу (Март 2023):

`<td align="left" width="33%"><a href="queries.html?date=2023-03-31&amp;period=month">&lt;&lt; Мар 23</a></td>`

Это ссылка, к которой добавляется дата - последнее число нужного месяца. То есть для получения данных за требуемые периоды, нужно научить парсер генерировать строку со ссылкой, в которой будет последовательно изменяться дата.

Создадим объект типа `Series` с интересующими датами для последующего перебора:

In [20]:
dates = np.array(['2023-04-30','2023-03-31', '2023-02-28', '2023-01-31',
                 '2022-12-31', '2022-11-30', '2022-10-31', '2022-09-30', '2022-08-31', '2022-07-31', '2022-06-30',
                 '2022-05-31', '2022-04-30', '2022-03-31', '2022-02-28', '2022-01-31',
                 '2021-12-31', '2021-11-30', '2021-10-31', '2021-09-30', '2021-08-31', '2021-07-31', '2021-06-30',
                 '2021-05-31', '2021-04-30', '2021-03-31', '2021-02-28', '2021-01-31'])

In [21]:
dates = pd.Series(dates)

Данные в столбце с датами строковые, что пока удобно, т.к. нужно будет подставлять их в строку. Но также потребуется создать два дополнительных столбца - с месяцем и годом, для того, чтобы можно было отслеживать динамику.  
Данные за апрель уже получены, но добавим к диапазону дату в таком же формате `2023-04-30`, чтобы получить все данные в одной таблице.

## Полный код парсера и сбор данных

Формируем пустой датафрейм, к которому будем присоединять таблицы:

In [22]:
query_stats_previous = pd.DataFrame()

Собираем парсер. В указанном диапазоне дат запускаем цикл, который будет последовательно подставлять и менять дату в адресе ссылки на дату из диапазона. Для каждой такой страницы будем получать html-структуру и находить в ней тег с номерами страниц. Затем по описанной ранее логике переберем все страницы с данными внутри выбранного месяца и сохраним таблицу с каждой страницы:

In [23]:
for i in dates:
#     ссылка на страницу с перебором дат и получение кода страницы
    url_previous = 'https://www.liveinternet.ru/stat/ru/queries.html?date={}&amp;period=month&total=yes&page='.format(i)
    r = rq.get(url_previous)
    soup = bs(r.text, 'html.parser')
    soup_pages = soup.find_all('a', class_='high')
#     список видимых страниц
    pages_num = []
#     перебор страниц и извлечение номера последней страницы в выбранном месяце
    for j in soup_pages:
        num = j.contents
        pages_num.append(num)
    last_page = pages_num[-2]
    last_page = [int(x) for x in last_page][0]
#     создание диапазона страниц внутри выбранного месяца
    pages = list(range(1,last_page+1))
#     получение таблицы с данными с каждой страницы
    for k in pages:
        url_pages = url_previous + str(k)
        stats = pd.read_html(url_pages, encoding='utf-8')[5]
#         добавление столбца с датой, извлечение месяца и года
        stats['period'] = pd.to_datetime(i)
        stats['month'] = stats['period'].dt.month
        stats['year'] = stats['period'].dt.year
#         соединение таблиц
        query_stats_previous = pd.concat([query_stats_previous,stats], ignore_index=True)

In [24]:
query_stats_previous.head()

,0,1,2,3,4,5,6,7,period,month,year
0,значения:суммарные / среднесуточные,значения:суммарные / среднесуточные,апрель 2023 г.,апрель 2023 г.,март 2023 г.,март 2023 г.,в среднемза 3 месяца,в среднемза 3 месяца,2023-04-30,4,2023
1,NaN,Другие,11065112,92.2%,27391283,89.5%,21787331,91.2%,2023-04-30,4,2023
2,NaN,новости,71069,0.6%,212279,0.7%,174103,0.7%,2023-04-30,4,2023
3,NaN,газета чс последний номер,36029,0.3%,84210,0.3%,99340,0.4%,2023-04-30,4,2023
4,NaN,майл ру,32251,0.3%,81623,0.3%,60865,0.3%,2023-04-30,4,2023


Установим первую строку в качестве наименований столбцов и сбросим индексы:

In [25]:
query_stats_previous = query_stats_previous.rename(columns=query_stats_previous.iloc[0])

In [26]:
query_stats_previous = query_stats_previous.reindex(query_stats_previous.index.drop(0))

In [27]:
query_stats_previous.head()

,значения:суммарные / среднесуточные,значения:суммарные / среднесуточные,апрель 2023 г.,апрель 2023 г.,март 2023 г.,март 2023 г.,в среднемза 3 месяца,в среднемза 3 месяца,2023-04-30 00:00:00,4,2023
1,NaN,Другие,11065112,92.2%,27391283,89.5%,21787331,91.2%,2023-04-30,4,2023
2,NaN,новости,71069,0.6%,212279,0.7%,174103,0.7%,2023-04-30,4,2023
3,NaN,газета чс последний номер,36029,0.3%,84210,0.3%,99340,0.4%,2023-04-30,4,2023
4,NaN,майл ру,32251,0.3%,81623,0.3%,60865,0.3%,2023-04-30,4,2023
5,NaN,майл,29342,0.2%,83063,0.3%,62428,0.3%,2023-04-30,4,2023


In [28]:
query_stats_previous.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3235 entries, 1 to 3235
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   значения:суммарные / среднесуточные  222 non-null    object        
 1   значения:суммарные / среднесуточные  2789 non-null   object        
 2   апрель 2023 г.                       2789 non-null   object        
 3   апрель 2023 г.                       2566 non-null   object        
 4   март 2023 г.                         2789 non-null   object        
 5   март 2023 г.                         2566 non-null   object        
 6   в среднемза 3 месяца                 2789 non-null   object        
 7   в среднемза 3 месяца                 2566 non-null   object        
 8   2023-04-30 00:00:00                  3235 non-null   datetime64[ns]
 9   4                                    3235 non-null   int64         
 10  2023        

Сохраняем полученный датасет для дальнейшего исследования:

In [29]:
query_stats_previous.to_csv('query_stats.csv')

Датасет готов, после предобработки можно использовать его для анализа.